In [8]:
import json
import os

import pandas as pd
import pdfplumber

os.getcwd()

'c:\\Users\\joost\\ReposWindows\\bedrock-web\\sandbox'

## Extract Data from .pdf with ChatGPT

Upload the AGS3-3-1-2005.pdf

Then use a prompt like this:

> Put the Groups and Headings table in a CSV with columns group_name,contents,notes,parent_group  
> The Groups and Headings table starts at the bottom of page 13 and finishes on page 17.

It might require a bit of chatting with ChatGPT before you get a proper table in your chat. Once you do, you can copy-paste it to a .tsv file. A .tsv file is a Tab Separated Value file instead of CSV - Comma Separated Value file, which is what you get when you copy the table from ChatGPT. TSV's are also handy, because they don't run into issues when one of the values contains a comma...

Now it could be that ChatGPT doesn't always return the same results and that the table isn't exactly as in the AGS 3 .pdf document. This was the case for me:

In [9]:
df1 = pd.read_csv("ags3_chatgpt_groups_and_headings.tsv", sep="\t")
df1["df"] = 1
df2 = pd.read_csv("ags3_chatgpt_groups_and_headings2.tsv", sep="\t")
df2["df"] = 2

# Concatenate the two DataFrames
df_concat = pd.concat([df1, df2])

# Find the duplicate rows
manual_duplicates = df_concat.duplicated(
    subset=df_concat.columns.difference(["df"]), keep=False
)

# Find the rows that are not duplicates (i.e., the rows that are unique to one DataFrame)
unique_rows = df_concat[~manual_duplicates]

unique_rows.sort_index()

,group_name,contents,parent_group,df
32,HPGI,Horizontal Profile Gauge Installation Details,HOLE,1
32,HPGI,Horizontal Profile Gauge Installation,HOLE,2
37,IFID,On Site Volatile Headspace Testing (FID),HOLE,2
37,IFID,On Site Volatile Headspace Testing Using Flame...,HOLE,1
38,INST,Single Point Instrument Installation,HOLE,2
38,INST,Single Point Instrument Installation Details,HOLE,1
40,IPID,On Site Volatile Headspace Testing (PID),HOLE,2
71,UNIT,Definition of <UNITS> and CNMT_UNIT,-,1
72,UNIT,Definition of Units,-,2


## Extract AGS 3 and 4 Data Dictionaries from their corresponding AGS .pdf documents

In [10]:
def extract_ags3_data_dict_table(table):
    headings = []
    for row in table[2:]:  # Skip first 2 rows: 1st = title, 2nd = headings
        headings.append(
            {
                "status": None if row[0] == "" else row[0].strip(),
                "heading": row[1].strip(),
                "unit": None if row[2] == "" else row[2].strip().replace("\n", " "),
                "description": row[3].strip().replace("\n", " "),
                "example": None if row[4] == "" else row[4].strip().replace("\n", " "),
            }
        )
    return headings


def extract_ags4_data_dict_table(table):
    # Skip rows that don't contain data
    for i, row in enumerate(table):
        if "Suggested\nUnit / Type" in row or "Unit / Type" in row:
            first_data_row = i + 1
            break

    headings = []
    for row in table[first_data_row:]:
        row = [x for x in row if x is not None]
        headings.append(
            {
                "status": None if row[0] == "" else row[0].strip(),
                "heading": row[1].strip(),
                "unit": None if row[2] == "" else row[2].strip().replace("\n", ""),
                "type": row[3].strip(),
                "description": row[4].strip().replace("\n", " "),
                "example": None if row[5] == "" else row[5].strip().replace("\n", " "),
            }
        )
    return headings

In [11]:
ags_version = 4  # AGS version 3 or 4
pdf_dict = {
    3: {"pdf_file": "AGS3_v3-1-2005.pdf", "from_page": 22, "to_page": 69},
    4: {"pdf_file": "AGS4-v4-1-1-2022.pdf", "from_page": 18, "to_page": 160},
}


In [12]:
pdf_file, from_page, to_page = pdf_dict[ags_version].values()

# List to store extracted data for each group
extracted_data = []
previous_group_name = ""
with pdfplumber.open(pdf_file) as pdf:
    # Adjust the page range based on where the tables are located
    for page_number in range(from_page, to_page):
        page = pdf.pages[page_number - 1]  # pdfplumber is 0-based, so subtract 1
        tables_on_current_page = page.extract_tables()  # Extract tables from the page

        # Iterate through all tables found on the page
        for table in tables_on_current_page:
            if ags_version == 3:
                table_title = table[0][0].strip()  # Get table title from AGS3
            elif ags_version == 4:
                table_title = table[0][1].strip()  # Get table title from AGS4
            print(table_title)

            parts = table_title.split(": ", 1)  # Split on the first occurrence of ': '
            if "Group Name" in parts[0]:
                group_name = parts[1].split(" - ")[0]
                group_description = " - ".join(parts[1].split(" - ")[1:])
                group_description = group_description.replace("\n", " ")
                if ags_version == 3:
                    headings = extract_ags3_data_dict_table(table)
                elif ags_version == 4:
                    headings = extract_ags4_data_dict_table(table)

                if group_name == previous_group_name:
                    extracted_data[-1]["headings"].extend(headings)
                else:
                    extracted_data.append(
                        {
                            "group_name": group_name,
                            "group_description": group_description,
                            "headings": headings,
                        }
                    )
                previous_group_name = group_name


Group Name: PROJ - Project Information
Group Name: ABBR - Abbreviation Definitions
Group Name: DICT - User Defined Groups and Headings
Group Name: FILE - Associated Files
Group Name: TRAN - Data File Transmission Information / Data Status
Group Name: TYPE - Definition of Data Types
Group Name: UNIT - Definition of Units
Group Name: AAVT - Aggregate Abrasion Tests
Group Name: AAVT - Aggregate Abrasion Tests
Group Name: ACVT - Aggregate Crushing Value Tests
Group Name: AELO - Aggregate Elongation Index Tests
Group Name: AFLK - Aggregate Flakiness Tests
Group Name: AFLK - Aggregate Flakiness Tests
Group Name: AIVT - Aggregate Impact Value Tests
Group Name: AIVT - Aggregate Impact Value Tests
Group Name: ALOS - Los Angeles Abrasion Tests
Group Name: ALOS - Los Angeles Abrasion Tests
Group Name: APSV - Aggregate Polished Stone Tests
Group Name: ARTW - Aggregate Determination of the Resistance to Wear (micro-Deval)
Group Name: ASDI - Slake Durability Index Tests
Group Name: ASNS - Aggregate 

In [13]:
with open(f"ags{ags_version}_manual_groups.json", "r") as f:
    manual_groups = json.load(f)

extracted_groups = [d["group_name"] for d in extracted_data]

print(set(manual_groups) - set(extracted_groups))
print(set(extracted_groups) - set(manual_groups))


{'LFCN'}
{'LFCN – Laboratory Fall Cone Test'}


In [14]:
# Save the extracted data to a JSON file
with open(f"ags{ags_version}_data_dict_p{from_page}-{to_page}.json", "w") as json_file:
    json.dump(extracted_data, json_file, indent=2)